This analysis presents the heart rate data from five mock audition sessions. Each singer performed two songs — one prepared, one assigned 24 hours in advance — for a three-person faculty panel. All subjects wore Polar H10 heart rate monitors. HR was recorded at 1 Hz (one value per second) and aligned to audio using a clap-based sync point.

**Participants:**

- Singer 1 (S1, M)
- Singer 2 (S2, F)
- Singer 3 (S3, F)
- Singer 4 (S4, F)
- Singer 5 (S5, M)
- Panelist 1 (F1) — HR data available for all 5 sessions
- Panelist 3 (F3) — HR data available for all 5 sessions

**Assigned pieces:** Female singers (S2, S3, S4): "If You Knew My Story" | Male singers (S1, S5): "Shiksa Goddess"

**Data constraint:** HR was exported from Polar Flow at 1 Hz — averaged BPM per second, not beat-to-beat RR intervals. Standard HRV metrics cannot be computed. Analysis focuses on HR magnitude, reactivity, and interpersonal synchrony.

In [1]:
#| echo: false
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Load data
hr = pd.read_csv('data/aligned_hr.csv')
phase_sum = pd.read_csv('data/phase_summaries.csv')

# Session metadata — anonymized
singer_names = {
    'S1': 'S1 (M)',
    'S2': 'S2 (F)',
    'S3': 'S3 (F)',
    'S4': 'S4 (F)',
    'S5': 'S5 (M)'
}

# Phase display order and colors
phase_order = ['pre_singing', 'song_1', 'inter_song_gap', 'song_2', 'post_session']
phase_labels = {
    'pre_singing': 'Pre-Singing',
    'song_1': 'Song 1 (Prepared)',
    'inter_song_gap': 'Inter-Song Gap',
    'song_2': 'Song 2 (Assigned)',
    'post_session': 'Post-Session'
}
phase_colors = {
    'pre_singing': 'rgba(255, 193, 7, 0.15)',
    'song_1': 'rgba(76, 175, 80, 0.15)',
    'inter_song_gap': 'rgba(158, 158, 158, 0.15)',
    'song_2': 'rgba(33, 150, 243, 0.15)',
    'post_session': 'rgba(244, 67, 54, 0.15)'
}

# Subject trace colors
trace_colors = {
    'singer': '#e74c3c',   # red
    'F1': '#8e44ad',       # purple
    'F3': '#2980b9'        # blue
}

In [2]:
#| echo: false
def make_session_figure(session_id, singer_label):
    """Create an interactive HR time series for one session."""
    sess = hr[hr['session'] == session_id]
    singer_id = session_id  # Singer ID matches session ID
    
    fig = go.Figure()
    
    # Add phase shading
    for phase in phase_order:
        phase_data = sess[sess['phase'] == phase]
        if len(phase_data) == 0:
            continue
        x0 = phase_data['elapsed_s'].min()
        x1 = phase_data['elapsed_s'].max()
        fig.add_vrect(
            x0=x0, x1=x1,
            fillcolor=phase_colors[phase],
            layer='below',
            line_width=0,
            annotation_text=phase_labels[phase],
            annotation_position='top left',
            annotation_font_size=10,
            annotation_font_color='#666'
        )
    
    # Singer trace
    singer_data = sess[sess['subject'] == singer_id]
    fig.add_trace(go.Scatter(
        x=singer_data['elapsed_s'],
        y=singer_data['hr_bpm'],
        mode='lines',
        name=singer_label,
        line=dict(color=trace_colors['singer'], width=2),
        hovertemplate='%{y} bpm<extra>' + singer_label + '</extra>'
    ))
    
    # Panelist 1 trace
    f1_data = sess[sess['subject'] == 'F1']
    fig.add_trace(go.Scatter(
        x=f1_data['elapsed_s'],
        y=f1_data['hr_bpm'],
        mode='lines',
        name='Panelist 1',
        line=dict(color=trace_colors['F1'], width=1.5),
        hovertemplate='%{y} bpm<extra>Panelist 1</extra>'
    ))
    
    # Panelist 3 trace
    f3_data = sess[sess['subject'] == 'F3']
    fig.add_trace(go.Scatter(
        x=f3_data['elapsed_s'],
        y=f3_data['hr_bpm'],
        mode='lines',
        name='Panelist 3',
        line=dict(color=trace_colors['F3'], width=1.5),
        hovertemplate='%{y} bpm<extra>Panelist 3</extra>'
    ))
    
    fig.update_layout(
        title=dict(text=f'Session {session_id} — Singer {session_id[1]} {singer_label}', font_size=16),
        xaxis_title='Elapsed Time (seconds)',
        yaxis_title='Heart Rate (bpm)',
        hovermode='x unified',
        template='plotly_white',
        height=400,
        legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1),
        margin=dict(t=80, b=60)
    )
    
    return fig

## Part 1: Individual Session Time Series

Each figure shows one audition session with the singer's HR (red) overlaid with Panelist 1's HR (purple) and Panelist 3's HR (blue). Shaded bands indicate audition phases. The x-axis shows elapsed time from the sync clap.

**Panelist baseline context:** Panelist 1 runs consistently higher than Panelist 3 across all sessions — typically 98–108 bpm vs. F3's 75–95 bpm. F1 is also more stable, with less variation across phases. The ~15–20 bpm gap between the two panelists is present in every session, suggesting a stable individual difference in resting autonomic tone rather than a differential response to specific singers.

### Session S1 — Singer 1 (M)

Singer 1 begins around 127 bpm and ramps steadily upward through song 1 (prepared piece), reaching peaks near 160 bpm. There is a brief dip during the inter-song gap, followed by a sharp rise into song 2 (assigned piece), where HR reaches its highest sustained values (~155–158 bpm). Panelist 1 holds steady around 100–108 bpm; Panelist 3 remains lower at 85–95 bpm. Both panelists are flat and stable — the singer's activation gap above the panelist mean is ~50–60 bpm during performance.

In [3]:
#| echo: false
fig_s1 = make_session_figure('S1', singer_names['S1'])
fig_s1.show()

### Session S2 — Singer 2 (F)

Singer 2 shows the steepest ramp-up of any singer — HR climbs from ~120 bpm at the start to peak near 169 bpm during song 1 (prepared). HR drops modestly during the inter-song gap (~145 bpm) but rebounds during song 2 (assigned), reaching ~155 bpm. Panelist 1 starts near 108 bpm and drifts down to ~100 bpm during song 1, then rises back toward 105 bpm in the post-session. Panelist 3 hovers lower in the 75–90 bpm range. The activation gap between singer and panelists widens as the session progresses.

In [4]:
#| echo: false
fig_s2 = make_session_figure('S2', singer_names['S2'])
fig_s2.show()

### Session S3 — Singer 3 (F)

Singer 3 follows a similar arc to Singer 2 — gradual climb through the pre-singing phase, sustained elevation during song 1 (135–165 bpm), partial recovery in the gap, and a plateau during song 2 (~150–158 bpm). Panelist 1 is steady around 101–107 bpm; Panelist 3 is lower at 80–95 bpm. The inter-song gap shows a brief but noticeable HR spike in Panelist 3's trace — likely anticipatory activation as the panelist prepares for the next performance.

In [5]:
#| echo: false
fig_s3 = make_session_figure('S3', singer_names['S3'])
fig_s3.show()

### Session S4 — Singer 4 (F)

Singer 4 is a clear outlier. HR starts around 80–90 bpm in the pre-singing phase — within the range typically seen in a *panelist*, not a singer. During song 1, HR rises modestly to ~105–120 bpm, well below the 140–170 bpm range seen in other singers. During song 2, HR actually *drops below baseline*, reaching the mid-70s. This is the most visually striking session in the dataset: all three traces (singer, F1, F3) occupy the same vertical range (~75–115 bpm), with frequent crossing and intertwining. Panelist 1 holds around 97–103 bpm but spikes to ~113 bpm during the inter-song gap — the highest F1 variability in any session (sd=5.8). Panelist 3 is similarly variable (sd=8.1 during the gap), at times crossing above the singer's HR.

**Important context:** Panelist 3 had a pre-existing teaching relationship with Singer 4. This likely explains the reduced anxiety response.

In [6]:
#| echo: false
fig_s4 = make_session_figure('S4', singer_names['S4'])
fig_s4.show()

### Session S5 — Singer 5 (M)

Singer 5 shows the highest sustained HR of any participant. Starting from a pre-singing baseline of ~135–140 bpm (already elevated), HR climbs rapidly through song 1 to peaks near 184 bpm. The inter-song gap provides minimal recovery (~165 bpm), and song 2 drives HR to its highest sustained values (~170–180 bpm). Panelist 1 stays in the 96–106 bpm range; Panelist 3 is lower at 73–85 bpm. The widest activation gap in the study occurs here — ~80 bpm between singer and panelist mean during song 2.

In [7]:
#| echo: false
fig_s5 = make_session_figure('S5', singer_names['S5'])
fig_s5.show()

## Part 2: Cross-Session Comparisons

### Singer HR by Audition Phase

The grouped bar chart below shows each singer's mean HR (with standard deviation bars) during the four performance phases. The pattern is consistent across four of five singers: HR is lowest during pre-singing, rises sharply for song 1, dips slightly during the inter-song gap, and remains elevated or continues rising into song 2. Singer 4 is the exception — her HR stays in the 88–106 bpm range across all phases, never approaching the 140+ bpm levels seen in the others.

**Key observation:** Pre-singing baseline HR is remarkably similar across Singers 1, 2, 3, and 5 (~126–143 bpm), suggesting comparable levels of anticipatory anxiety before singing begins. Singer 4's pre-singing HR (~91 bpm) is 35–50 bpm lower than the group.

In [8]:
#| echo: false
# Phase summary bar chart — singer HR by phase
singers = ['S1', 'S2', 'S3', 'S4', 'S5']
perf_phases = ['pre_singing', 'song_1', 'inter_song_gap', 'song_2']

phase_bar_colors = {
    'pre_singing': '#FFC107',
    'song_1': '#4CAF50',
    'inter_song_gap': '#9E9E9E',
    'song_2': '#2196F3'
}

fig_phase = go.Figure()

for phase in perf_phases:
    means = []
    sds = []
    labels = []
    for s in singers:
        row = phase_sum[(phase_sum['session'] == s) & 
                        (phase_sum['subject'] == s) & 
                        (phase_sum['phase'] == phase)]
        if len(row) > 0:
            means.append(row['mean_hr'].values[0])
            sds.append(row['sd_hr'].values[0])
        else:
            means.append(0)
            sds.append(0)
        labels.append(singer_names[s])
    
    fig_phase.add_trace(go.Bar(
        name=phase_labels[phase],
        x=labels,
        y=means,
        error_y=dict(type='data', array=sds, visible=True),
        marker_color=phase_bar_colors[phase],
        hovertemplate='%{y:.1f} ± %{error_y.array:.1f} bpm<extra>' + phase_labels[phase] + '</extra>'
    ))

fig_phase.update_layout(
    title=dict(text='Singer Mean HR by Audition Phase', font_size=16),
    xaxis_title='Singer',
    yaxis_title='Heart Rate (bpm)',
    barmode='group',
    template='plotly_white',
    height=450,
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1),
    margin=dict(t=80, b=60)
)

fig_phase.show()

### Singer–Panelist Activation Gap

The chart below quantifies the HR difference between each singer and the mean of available panelists (Faculty 1 and Faculty 3) for each phase. The gap reflects how far the singer's autonomic activation exceeds the panelists' — a rough index of the anxiety asymmetry in the audition relationship.

Singers 1, 2, 3, and 5 show gaps of 30–78 bpm, widening as the session progresses. The gap is largest during the inter-song gap and song 2 for most singers — suggesting that the stress of performing the assigned piece (less familiar, less rehearsed) compounds the already elevated state from song 1.

Singer 4's gap is negative in most phases — her HR falls *below* the panelist mean. This is the only singer whose physiological state approximates or drops beneath the panelists'.

In [9]:
#| echo: false
# Activation gap chart
fig_gap = go.Figure()

singer_colors = {
    'S1': '#e74c3c',
    'S2': '#e67e22',
    'S3': '#27ae60',
    'S4': '#3498db',
    'S5': '#8e44ad'
}

for s in singers:
    gaps = []
    for phase in perf_phases:
        singer_row = phase_sum[(phase_sum['session'] == s) & 
                               (phase_sum['subject'] == s) & 
                               (phase_sum['phase'] == phase)]
        f1_row = phase_sum[(phase_sum['session'] == s) & 
                           (phase_sum['subject'] == 'F1') & 
                           (phase_sum['phase'] == phase)]
        f3_row = phase_sum[(phase_sum['session'] == s) & 
                           (phase_sum['subject'] == 'F3') & 
                           (phase_sum['phase'] == phase)]
        if len(singer_row) > 0 and len(f1_row) > 0 and len(f3_row) > 0:
            panelist_mean = (f1_row['mean_hr'].values[0] + f3_row['mean_hr'].values[0]) / 2
            gap = singer_row['mean_hr'].values[0] - panelist_mean
            gaps.append(gap)
        else:
            gaps.append(0)
    
    fig_gap.add_trace(go.Bar(
        name=singer_names[s],
        x=[phase_labels[p] for p in perf_phases],
        y=gaps,
        marker_color=singer_colors[s],
        hovertemplate='%{y:+.1f} bpm<extra>' + singer_names[s] + '</extra>'
    ))

fig_gap.add_hline(y=0, line_dash='dash', line_color='black', line_width=1)

fig_gap.update_layout(
    title=dict(text='Singer–Panelist Activation Gap by Phase', font_size=16),
    xaxis_title='Phase',
    yaxis_title='HR Gap (singer − panelist mean, bpm)',
    barmode='group',
    template='plotly_white',
    height=450,
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1),
    margin=dict(t=80, b=60)
)

fig_gap.show()

### HR Reactivity Profiles

The dot-and-line plot below connects each singer's mean HR across the four phases, making the *shape* of each stress response visible.

Four distinct profiles emerge:

- **Singer 5** — highest absolute HR, steep climb, no recovery between songs. The most pronounced stress response in the study.
- **Singer 2** — steep initial climb during song 1, partial recovery in the gap, rebound for song 2. Classic performance anxiety arc.
- **Singers 1 and 3** — moderate, steady climb. Singer 1 shows the most gradual escalation; Singer 3 plateaus earlier.
- **Singer 4** — flat trajectory near 90 bpm. Modest rise during song 1, return to baseline for song 2. This profile resembles a panelist more than a performer.

In [10]:
#| echo: false
# Reactivity profiles — dot-and-line
fig_react = go.Figure()

for s in singers:
    means = []
    for phase in perf_phases:
        row = phase_sum[(phase_sum['session'] == s) & 
                        (phase_sum['subject'] == s) & 
                        (phase_sum['phase'] == phase)]
        if len(row) > 0:
            means.append(row['mean_hr'].values[0])
        else:
            means.append(None)
    
    fig_react.add_trace(go.Scatter(
        x=[phase_labels[p] for p in perf_phases],
        y=means,
        mode='lines+markers',
        name=singer_names[s],
        line=dict(color=singer_colors[s], width=2),
        marker=dict(size=10),
        hovertemplate='%{y:.1f} bpm<extra>' + singer_names[s] + '</extra>'
    ))

fig_react.update_layout(
    title=dict(text='Singer HR Reactivity Profiles Across Phases', font_size=16),
    xaxis_title='Phase',
    yaxis_title='Mean Heart Rate (bpm)',
    template='plotly_white',
    height=450,
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1),
    margin=dict(t=80, b=60)
)

fig_react.show()

## Summary

| Singer | Pre-Singing HR | Song 1 HR | Song 2 HR | Reactivity (Song 1 − Baseline) | Activation Gap (Song 2) | Notes |
|--------|---------------|-----------|-----------|-------------------------------|------------------------|-------|
| S1 (M) | 128 | 144 | 154 | +16 | +56 | Steady climber; flattest assigned-song trace |
| S2 (F) | 128 | 160 | 147 | +32 | +54 | Steepest ramp-up; HR *drops* for song 2 |
| S3 (F) | 126 | 150 | 152 | +24 | +59 | Moderate, steady; similar to S2 during assigned song |
| S4 (F) | 91 | 106 | 90 | +15 | −5 | Outlier — pre-existing relationship with F3; HR *below* panelist mean |
| S5 (M) | 143 | 167 | 170 | +24 | +80 | Highest absolute HR; no recovery between songs |

*All values are mean HR in bpm. Activation gap = singer mean HR minus mean of Panelist 1 and Panelist 3 during song 2.*